In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as spst

from ipywidgets import widgets
!pip install plotly
import plotly.graph_objs as go 
%matplotlib inline

In [2]:

p_value=widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=1.0,
    step=0.01,
    disabled=False,
    discription="p",
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

steps=widgets.ToggleButtons(
   # value=False,
    #description='Click me',
   # disabled=False,
    options=["complete PMF","decompose"],
    #button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

x=np.arange(20)
def pmf_update(p):
    x=np.arange(20)
    pmf_1=spst.poisson.pmf(x,1)
    pmf_2=spst.poisson.pmf(x,10)
    pmf=pmf_1*p+pmf_2*(1-p)
    return pmf, pmf_1*p,pmf_2*(1-p)
pmf,pmf1w,pmf2w=pmf_update(0)

trace1=go.Bar(x=x,y=pmf, 
              hoverinfo="text",text="",name="mixed PMF"+" "*10 , width=0.8 )
trace2=go.Bar(x=x,y=pmf1w, 
              hoverinfo="text",text="",name="PMF1 component"  , width=0.8 )
trace3=go.Bar(x=x,y=pmf2w, 
              hoverinfo="text",text="",name="PMF2 component"  , width=0.8 )

g = go.FigureWidget(data=[trace1],
                        layout=go.Layout(
                        xaxis = dict(title = " x values", range = [-0.5,20.5],showticklabels=True),
                        yaxis = dict(title = "probability", range = [-.1,0.4]),
                        hovermode=None,
                        margin={'l': 0, 'r': 0, 't': 50, 'b': 0},width=400, height=300  ),
                   )
g.update_layout(barmode="stack",
                showlegend=False,
                title=dict(text='mixture distribution', x=0.5,y=0.9))
def response1(change):
    pmf,pmf1w,pmf2w=pmf_update(p_value.value)
    if steps.value=="complete PMF":
        with g.batch_update():
            g.data[0].y=pmf
    else:
        with g.batch_update():
            g.data[0].y=pmf1w
            g.data[1].y=pmf2w

def response2(change):
    pmf,pmf1w,pmf2w=pmf_update(p_value.value)
    if steps.value=="complete PMF":
        with g.batch_update():
            g.data=[g.data[0]]
            response1(change)
    else:
        with g.batch_update():
            g.data=[]
            trace2.y=pmf1w
            trace3.y=pmf2w
            g.add_trace(trace2)
            g.add_trace(trace3)

container1 = widgets.VBox([p_value,steps]) 
p_value.observe(response1,names="value")
steps.observe(response2,names="value")

Widget1=widgets.VBox([container1,g] )
Widget1
